In [1]:
import numpy as np
from darli.models import RobotModel
import casadi as cs

In [2]:
urdf_file = '../yet_another_menagerie/z1/z1.urdf'
model = RobotModel(urdf_file, bodies_names={'ee': 'link06'})

In [3]:
opti_problem = cs.Opti()
q = opti_problem.variable(model.nq)

In [4]:
pos = model.body('ee').position(q)
gravity = model.inverse_dynamics(q, 0, 0)

In [5]:
# TODO: move to parameters
desired_pos = opti_problem.parameter(3)


In [6]:
cost = cs.sumsqr(pos - desired_pos) + 0.001*cs.sumsqr(gravity)
opti_problem.minimize(cost)
opti_problem.subject_to(opti_problem.bounded(model.q_min, q, model.q_max))
opti_problem.solver('ipopt')

In [7]:
opti_problem.set_value(desired_pos,[0.2, 0.2, 0.36])

In [8]:
opti_problem.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 6)
  #parameters: 1 (np = 3)
  #constraints: 1 (ng = 6)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [9]:
q_opt = opti_problem.value(q)
q_opt

array([ 7.85397669e-01,  1.39999804e+00, -1.25671438e+00,  9.70957982e-01,
        2.15841899e-06,  1.20044243e-01])

In [10]:
np.array(model.body('ee').position(q_opt))

array([[0.15341843],
       [0.15341842],
       [0.36670296]])

In [11]:
# opti_problem.solver('sqpmethod')
opti_problem.set_initial(q,q_opt)
opti_problem.set_value(desired_pos,[0.19, 0.21, 0.37])
opti_problem.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.8785899e-02 0.00e+00 3.07e-03  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 6)
  #parameters: 1 (np = 3)
  #constraints: 1 (ng = 6)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})